In [1]:
from ns import ns

In [2]:
networks = (1,2,3)
terminals = (2,3,4)
node_containers = []

bridge_device_containers = [ns.internet.NetDeviceContainer() for _ in range(len(networks))]
net_device_containers = [ns.internet.NetDeviceContainer() for _ in range(len(networks))]

switch_container = ns.network.NodeContainer()
switch_container.Create(len(networks))
router_node_container = ns.network.NodeContainer()
router_node_container.Create(1)

In [3]:
for terminal_count in terminals:
    node_container = ns.network.NodeContainer()
    node_container.Create(terminal_count)
    node_containers.append(node_container)

In [4]:
node_containers[2].GetN()

4

In [5]:
csma = ns.csma.CsmaHelper()

In [6]:
for idx, node_container in enumerate(node_containers):
    for node_idx in range(node_container.GetN()):
        csma_node_pair = ns.network.NodeContainer()
        csma_node_pair.Add(node_container.Get(node_idx))
        csma_node_pair.Add(switch_container.Get(idx))
        devices = csma.Install(csma_node_pair)
        bridge_device_containers[idx].Add(devices.Get(1))
        net_device_containers[idx].Add(devices.Get(0))
    csma_node_pair = ns.network.NodeContainer()
    csma_node_pair.Add(switch_container.Get(idx))
    csma_node_pair.Add(router_node_container.Get(0))
    devices = csma.Install(csma_node_pair)
    bridge_device_containers[idx].Add(devices.Get(0))
    net_device_containers[idx].Add(devices.Get(1))

In [7]:
bridge_helper = ns.bridge.BridgeHelper()
for idx, bridge_device_container in enumerate(bridge_device_containers):
    bridge_helper.Install(switch_container.Get(idx), bridge_device_container)

In [8]:
ip_stack = ns.internet.InternetStackHelper()
for node_container in node_containers:
    ip_stack.Install(node_container)
ip_stack.Install(router_node_container)

In [9]:
addr = ns.internet.Ipv4AddressHelper()
addr.SetBase(ns.network.Ipv4Address(f"10.0.0.0"), ns.network.Ipv4Mask("255.255.255.0"))

for net_device_container in net_device_containers:
    address_container = addr.Assign(net_device_container)
    addr.NewNetwork()

In [10]:
from lib.utils import get_node_ips, get_node_ips_all

In [11]:
addresses = get_node_ips_all(node_containers[0], verbose=True)

{
  "0": [
    "m_local=127.0.0.1; m_mask=255.0.0.0; m_broadcast=127.255.255.255; m_scope=0; m_secondary=0"
  ],
  "1": [
    "m_local=10.0.0.1; m_mask=255.255.255.0; m_broadcast=10.0.0.255; m_scope=2; m_secondary=0"
  ]
}
{
  "0": [
    "m_local=127.0.0.1; m_mask=255.0.0.0; m_broadcast=127.255.255.255; m_scope=0; m_secondary=0"
  ],
  "1": [
    "m_local=10.0.0.2; m_mask=255.255.255.0; m_broadcast=10.0.0.255; m_scope=2; m_secondary=0"
  ]
}


In [12]:
ns.internet.Ipv4GlobalRoutingHelper.PopulateRoutingTables()

In [13]:
local_servers = ns.network.NodeContainer()
local_clients = ns.network.NodeContainer()
remote_servers = ns.network.NodeContainer()
remote_clients = ns.network.NodeContainer()

for idx, network_id in enumerate(networks):
    terminal_count = node_containers[idx].GetN()
    local_servers.Add(node_containers[idx].Get(terminal_count - 1))
    local_clients.Add(node_containers[idx].Get(0))
    if idx in [0,1]:
        remote_clients.Add(node_containers[idx].Get(0))
    # if idx in [1,2]:
    #     remote_servers.Add(node_containers[idx].Get(terminal_count-1))

In [14]:
ECHO_PORT = 9
echo_srv_helper = ns.UdpEchoServerHelper(ECHO_PORT)
srv_apps = echo_srv_helper.Install(local_servers)
print(f"Servers: {srv_apps.GetN()}")

Servers: 3


In [15]:
from lib.utils import create_echo_client_helper

client_apps = ns.applications.ApplicationContainer()

for idx, network_id in enumerate(networks):
    server_address = get_node_ips(local_servers.Get(idx))[1][0].GetAddress()
    server_address = server_address.ConvertTo()
    echo_client_helper = create_echo_client_helper(server_address, ECHO_PORT)
    client_app = echo_client_helper.Install(local_clients.Get(idx))
    client_apps.Add(client_app)

# inter-network communication
# client 0 -> server 1
server_address = get_node_ips(local_servers.Get(1))[1][0].GetAddress()
server_address = server_address.ConvertTo()
echo_client_helper = create_echo_client_helper(server_address, ECHO_PORT)
client_app = echo_client_helper.Install(local_clients.Get(0))
client_apps.Add(client_app)

# client 1 -> server 2
server_address = get_node_ips(local_servers.Get(2))[1][0].GetAddress()
server_address = server_address.ConvertTo()
echo_client_helper = create_echo_client_helper(server_address, ECHO_PORT)
client_app = echo_client_helper.Install(local_clients.Get(1))
client_apps.Add(client_app)

In [16]:
client_apps.GetN()

5

## Time Simulation

In [17]:
srv_apps.Start(ns.core.Seconds(1.0))
srv_apps.Stop(ns.core.Seconds(30.0))
client_apps.Start(ns.core.Seconds(2.0)) # has to be later than server
client_apps.Stop(ns.core.Seconds(30.0))

## Logging

In [18]:
ns.LogComponentEnable("UdpEchoClientApplication", ns.core.LOG_LEVEL_INFO)
ns.LogComponentEnable("UdpEchoServerApplication", ns.core.LOG_LEVEL_INFO)

## Run Simulation

In [19]:
ns.Simulator.Stop(ns.Seconds(35))
ns.Simulator.Run()
ns.Simulator.Destroy()

At time +2s client sent 1500 bytes to 10.0.0.2 port 9
At time +2s client sent 1500 bytes to 10.0.1.3 port 9
At time +2s client sent 1500 bytes to 10.0.1.3 port 9
At time +2s client sent 1500 bytes to 10.0.2.4 port 9
At time +2s client sent 1500 bytes to 10.0.2.4 port 9
At time +2.00201s server received 1500 bytes from 10.0.2.1 port 49153
At time +2.00201s server sent 1500 bytes to 10.0.2.1 port 49153
At time +2.00402s server received 1500 bytes from 10.0.0.1 port 49154
At time +2.00402s server sent 1500 bytes to 10.0.0.1 port 49154
At time +2.00601s server received 1500 bytes from 10.0.0.1 port 49153
At time +2.00601s server sent 1500 bytes to 10.0.0.1 port 49153
At time +2.00802s client received 1500 bytes from 10.0.2.4 port 9
At time +2.00901s server received 1500 bytes from 10.0.1.1 port 49153
At time +2.00901s server sent 1500 bytes to 10.0.1.1 port 49153
At time +2.00903s client received 1500 bytes from 10.0.1.3 port 9
At time +2.01202s server received 1500 bytes from 10.0.1.1 por